In [1]:
#Idea: 

In [2]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


def clear_dataframe():
    return pd.DataFrame()

In [3]:
#Converting the dta files from American Economic Association Journal's Appendix (paper titled "Wells, Water, and Welfare: The Impact of Access to Groundwater on Rural Poverty and Conflict")

iac = pd.io.stata.read_stata(r'C:\Users\vishe\Desktop\Groundwater\AEJ_IAC.dta')
iac.to_csv(r'C:\Users\vishe\Desktop\Groundwater\AEJ_IAC.csv')

In [4]:

def aquifer_depth_label(dmaq2, dmaq3):
    if dmaq3 == 1:
        return 'high'
    elif dmaq2 == 1:
        return 'medium'
    else:
        return 'low'

# Create a new column 'aquifer_depth_str'
iac['aquifer_depth_str'] = iac.apply(lambda row: aquifer_depth_label(row['dmaq2'], row['dmaq3']), axis=1)




C:\Users\vishe\AppData\Local\Temp\ipykernel_21256\1550435982.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  iac['aquifer_depth_str'] = iac.apply(lambda row: aquifer_depth_label(row['dmaq2'], row['dmaq3']), axis=1)


In [5]:

def aquifer_depth(dmaq2, dmaq3):
    if dmaq3 == 1:
        return 3
    elif dmaq2 == 1:
        return 2
    else:
        return 1

# Create a new column 'aquifer_depth'
iac['aquifer_depth'] = iac.apply(lambda row: aquifer_depth(row['dmaq2'], row['dmaq3']), axis=1)


C:\Users\vishe\AppData\Local\Temp\ipykernel_21256\1694916534.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  iac['aquifer_depth'] = iac.apply(lambda row: aquifer_depth(row['dmaq2'], row['dmaq3']), axis=1)


In [6]:
iac.head

<bound method NDFrame.head of       year  code  dms02  dms03  dms04  dms05  dms06  dms07  dms08  dms09  ...  \
0     1956   112      0      0      0      0      0      0      0      0  ...   
1     1957   112      0      0      0      0      0      0      0      0  ...   
2     1958   112      0      0      0      0      0      0      0      0  ...   
3     1959   112      0      0      0      0      0      0      0      0  ...   
4     1960   112      0      0      0      0      0      0      0      0  ...   
...    ...   ...    ...    ...    ...    ...    ...    ...    ...    ...  ...   
3963  1983  1717      1      0      0      0      0      0      0      0  ...   
3964  1984  1717      1      0      0      0      0      0      0      0  ...   
3965  1985  1717      1      0      0      0      0      0      0      0  ...   
3966  1986  1717      1      0      0      0      0      0      0      0  ...   
3967  1987  1717      1      0      0      0      0      0      0      0  ...  

In [7]:
#Method 2: SVM

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Assuming 'iac' is your DataFrame
X = iac[['alt','dsea', 'at', 'pr']]  # Include your features
y = iac['aquifer_depth_str']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

# Print shapes of training and testing sets for debugging
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Create an SVM classifier
svm_classifier = SVC(kernel='linear', C=1)

# Train the classifier on the training set
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
#accuracy = accuracy_score(y_test, y_pred)
#print(f"Accuracy: {accuracy}")

print("accuarcy score is : ", accuracy_score(y_test, y_pred))


X_train shape: (2777, 4)
y_train shape: (2777,)
X_test shape: (1191, 4)
y_test shape: (1191,)
accuarcy score is :  0.6120906801007556


In [8]:
#Method 1: Generalised Linear Model

model = smf.glm(formula = "aquifer_depth ~ alt + dsea + at + pr", 
                data = iac, 
                family = sm.families.Binomial())

# Fit the model
result = model.fit()
# Display and interpret results
print(result.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:          aquifer_depth   No. Observations:                 3968
Model:                            GLM   Df Residuals:                     3963
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                   -inf
Date:                Wed, 27 Dec 2023   Deviance:                   2.7862e+05
Time:                        02:07:50   Pearson chi2:                 2.31e+19
No. Iterations:                     5   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.012e+16   4.17e+07  -7.22e+08      0.0

C:\Users\vishe\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\genmod\families\links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
C:\Users\vishe\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\genmod\families\family.py:1056: RuntimeWarning: invalid value encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
C:\Users\vishe\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\genmod\families\family.py:1057: RuntimeWarning: invalid value encountered in log
  n * np.log(1 - mu + 1e-20)) * var_weights
